In [8]:
import pandas as pd
import os
import time
import requests
import json
import csv
from tqdm import tqdm

import tweepy

import requests
import pandas as pd
import os
import collections
from langdetect import detect
import matplotlib.pyplot as plt

f = open("api-keys.json")
input=json.load(f)

In [38]:
import nltk
import string 
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

# GET STOPWORDS
from nltk.corpus import stopwords
stop_words=nltk.corpus.stopwords.words('english')

# INITALIZE STEMMER+LEMITZIZER+SIA
sia = SentimentIntensityAnalyzer()
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [9]:
# LOAD KEYS INTO API
consumer_key=input["consumer_key"]    
consumer_secret=input["consumer_secret"]    
access_token=input["access_token"]    
access_token_secret=input["access_token_secret"]    
bearer_token=input["bearer_token"]    


In [10]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)
headers = {"Authorization": "Bearer {}".format(bearer_token)}
print(api)
print(api.verify_credentials().screen_name)

monk_digi


In [12]:
def search_more(query,tweets_num):
    num_tweets_collected=0
    searches=[]
    while num_tweets_collected<tweets_num:
        if len(searches)==0:
            search_results = api.search_tweets(query,lang="en", count=100)
        else:
            search_results = api.search_tweets(query,lang="en", count=100,max_id=max_id_next)
        num_tweets_collected+=len(search_results)
        max_id_next=int(search_results[-1]._json["id_str"])-1

        for i in range(len(search_results)):
            searches.append(search_results[i])
    return(searches)

In [21]:
def sh_to_list(searches):
    texts=[]
    for i in range(len(searches)):
        texts.append(searches[i]._json["text"])
    return texts
sh_free_list=sh_to_list(sh_free)

In [30]:
#acquire data from twitter

sh_free=search_more("freeway",3000)
sh_free_list=list(set(sh_to_list(sh_free)))
sh_mtr=search_more("metro",3000)
sh_mtr_list=list(set(sh_to_list(sh_mtr)))

In [36]:
# clean the text data

def clean_string(text):
    #FILTER OUT UNWANTED CHAR
    new_text=""
    keep=" abcdefghijklmnopqrstuvwxyz0123456789"
    for character in text:
        if character.lower() in keep:
            new_text+=character.lower()
        else: 
            new_text+=" "
    text=new_text

    #FILTER OUT UNWANTED WORDS
    new_text=""
    for word in nltk.tokenize.word_tokenize(text):
        if word not in nltk.corpus.stopwords.words('english'):
            #lemmatize 
            tmp=lemmatizer.lemmatize(word)
            word=tmp
            if len(word)>1:
                if word in [".",",","!","?",":",";"]:
                    #remove the last space
                    new_text=new_text[0:-1]+word+" "
                else: #add a space
                    new_text+=word.lower()+" "
    text=new_text.strip()
    return text

In [39]:
for i in range(len(sh_free_list)):
    sh_free_list[i]=clean_string(sh_free_list[i])

In [41]:
for i in range(len(sh_mtr_list)):
    sh_mtr_list[i]=clean_string(sh_mtr_list[i])

In [49]:
tmp1=[]
for i in range(0,len(sh_free_list)):
    tmp1.append([sh_free_list[i],"freeway"])
df1=pd.DataFrame(tmp1)
df1=df1.rename(columns={0: "text", 1: "label"})
print(df1)

                                                   text    label
0                            thinking cat named freeway  freeway
1     keilor lodge heavy traffic bothways calder fre...  freeway
2     lot going funniest though literally person hor...  freeway
3     spanish lady corner selling fruit another sell...  freeway
4     leftcoastbias last one bakersfield hyatt house...  freeway
...                                                 ...      ...
1855  mckbkb one really loved riding harley car rush...  freeway
1856  rt smmirror upcoming play showcase event two g...  freeway
1857  port melbourne heavy traffic citybound west ga...  freeway
1858  galveston county 45 gulf freeway southbound fm...  freeway
1859  ever since found way take street amp house fre...  freeway

[1860 rows x 2 columns]


In [50]:
tmp2=[]
for i in range(0,len(sh_mtr_list)):
    tmp2.append([sh_mtr_list[i],"metro"])
df2=pd.DataFrame(tmp2)
df2=df2.rename(columns={0: "text", 1: "label"})
print(df2)

                                                   text  label
0     pdxfato hcp1hcp tinakotek true live west pdx m...  metro
1     thad calton local metro school easily accessib...  metro
2     bearnar33940319 hi please contact customer rel...  metro
3     tylerdaisydog1 metro sport tuchel manager main...  metro
4     rodentdetector brienn365 melissalogic17 nwarik...  metro
...                                                 ...    ...
1745  unsethled though said right completely true de...  metro
1746  sure hope replacement jewish trumpcrimefamily ...  metro
1747  elliotftbshirts brilliant metro easy enough tr...  metro
1748  rt profcanderson footage made clear pelosi sch...  metro
1749  rt sitetrend top metro area number startup new...  metro

[1750 rows x 2 columns]


In [53]:
df=pd.concat([df1,df2])
print(df)
df.to_csv('free_mtr_text_data.csv',index=False)

                                                   text    label
0                            thinking cat named freeway  freeway
1     keilor lodge heavy traffic bothways calder fre...  freeway
2     lot going funniest though literally person hor...  freeway
3     spanish lady corner selling fruit another sell...  freeway
4     leftcoastbias last one bakersfield hyatt house...  freeway
...                                                 ...      ...
1745  unsethled though said right completely true de...    metro
1746  sure hope replacement jewish trumpcrimefamily ...    metro
1747  elliotftbshirts brilliant metro easy enough tr...    metro
1748  rt profcanderson footage made clear pelosi sch...    metro
1749  rt sitetrend top metro area number startup new...    metro

[3610 rows x 2 columns]
